# Setup Verification Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Javihaus/agents_observability_bootcamp/blob/main/setup/04_verify_setup.ipynb)

This notebook verifies that your environment is correctly configured for the Agentic AI Systems Masterclass.

**Run all cells in order.** Each test must pass before proceeding to the next.

---

## What This Notebook Tests

1. **Google Colab Environment** - Runtime specifications and connectivity
2. **Package Installation** - LangChain, Anthropic, and dependencies
3. **API Key Configuration** - Claude API key in Colab Secrets
4. **API Connectivity** - Successful connection to Anthropic API
5. **Basic Agent Execution** - Simple LangChain agent workflow

**Expected runtime**: 2-3 minutes

---

## Instructions

1. Click **Runtime** → **Run all** (or press Ctrl+F9)
2. Watch for green checkmarks indicating success
3. If any test fails, follow the troubleshooting guidance
4. All tests must pass before Day 1

**If you see red X marks, stop and fix issues before continuing.**

---
## Test 1: Google Colab Environment

Verifies you're running in Google Colab with appropriate resources.

In [ ]:
import sys
import platform

print("Testing Google Colab environment...\n")

# Check if running in Colab
try:
    import google.colab
    print("✓ Running in Google Colab")
except ImportError:
    print("✗ NOT running in Google Colab")
    print("  Please open this notebook in Google Colab")
    print("  URL: https://colab.research.google.com/github/Javihaus/agents_observability_bootcamp/blob/main/setup/04_verify_setup.ipynb")
    sys.exit(1)

# Display system information
print(f"✓ Python version: {sys.version.split()[0]}")
print(f"✓ Platform: {platform.system()} {platform.release()}")

# Check Python version (must be 3.9+)
python_version = tuple(map(int, sys.version.split()[0].split('.')))
if python_version < (3, 9):
    print(f"\n✗ Python {sys.version.split()[0]} is too old")
    print("  Required: Python 3.9 or higher")
    print("  Contact instructor for assistance")
    sys.exit(1)

print("\n✓ Test 1 PASSED: Environment is compatible\n")

---
## Test 2: Package Installation

Installs and verifies all required packages.

In [ ]:
print("Installing required packages...\n")

# Install packages quietly
!pip install -q langchain==0.1.0
!pip install -q langchain-anthropic==0.1.1
!pip install -q anthropic==0.18.1
!pip install -q python-dotenv
!pip install -q pandas numpy matplotlib seaborn

print("✓ Packages installed\n")

In [ ]:
print("Verifying package versions...\n")

# Test core imports
try:
    import langchain
    print(f"✓ langchain: {langchain.__version__}")
    
    if langchain.__version__ != "0.1.0":
        print(f"  ⚠ Warning: Expected version 0.1.0, got {langchain.__version__}")
        print("  This may cause compatibility issues")
except ImportError as e:
    print(f"✗ Failed to import langchain: {e}")
    sys.exit(1)

try:
    import langchain_anthropic
    print(f"✓ langchain-anthropic installed")
except ImportError as e:
    print(f"✗ Failed to import langchain-anthropic: {e}")
    sys.exit(1)

try:
    import anthropic
    print(f"✓ anthropic: {anthropic.__version__}")
except ImportError as e:
    print(f"✗ Failed to import anthropic: {e}")
    sys.exit(1)

# Test utility imports
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    print("✓ Data science packages: pandas, numpy, matplotlib")
except ImportError as e:
    print(f"✗ Failed to import utility packages: {e}")
    sys.exit(1)

print("\n✓ Test 2 PASSED: All packages installed correctly\n")

---
## Test 3: API Key Configuration

Verifies your Claude API key is stored in Colab Secrets.

**If this test fails**, follow these steps:
1. Click the key icon (🔑) in the left sidebar
2. Click "Add new secret"
3. Name: `ANTHROPIC_API_KEY`
4. Value: Your API key from https://console.anthropic.com/settings/keys
5. Toggle "Notebook access" to ON
6. Click "Save"
7. Re-run this cell

In [ ]:
from google.colab import userdata

print("Checking API key configuration...\n")

try:
    # Attempt to retrieve API key
    api_key = userdata.get('ANTHROPIC_API_KEY')
    
    print("✓ API key found in Colab Secrets")
    
    # Validate key format
    if not api_key.startswith('sk-ant'):
        print("\n✗ API key format invalid")
        print("  Expected format: sk-ant-api03-...")
        print(f"  Your key starts with: {api_key[:10]}...")
        print("\n  Please verify you copied the correct key from Anthropic Console")
        sys.exit(1)
    
    if len(api_key) < 50:
        print("\n✗ API key seems too short")
        print(f"  Length: {len(api_key)} characters")
        print("  Expected: 100+ characters")
        print("\n  Please verify you copied the complete key")
        sys.exit(1)
    
    print(f"✓ Key format valid (prefix: {api_key[:15]}...)")
    print(f"✓ Key length: {len(api_key)} characters")
    
except userdata.SecretNotFoundError:
    print("✗ API key NOT found in Colab Secrets")
    print("\nSetup instructions:")
    print("1. Click the key icon (🔑) in the left sidebar")
    print("2. Click 'Add new secret'")
    print("3. Name: ANTHROPIC_API_KEY")
    print("4. Value: Your API key from https://console.anthropic.com/settings/keys")
    print("5. Toggle 'Notebook access' to ON")
    print("6. Click 'Save'")
    print("7. Re-run this cell")
    print("\nDetailed guide: https://github.com/Javihaus/agents_observability_bootcamp/blob/main/setup/01_claude_api_setup.md")
    sys.exit(1)

except userdata.NotebookAccessDisabledError:
    print("✗ API key exists but notebook access is disabled")
    print("\nFix:")
    print("1. Click the key icon (🔑) in the left sidebar")
    print("2. Find 'ANTHROPIC_API_KEY'")
    print("3. Toggle 'Notebook access' to ON")
    print("4. Re-run this cell")
    sys.exit(1)

except Exception as e:
    print(f"✗ Unexpected error: {e}")
    print("\nContact instructor: javier@jmarin.info")
    sys.exit(1)

print("\n✓ Test 3 PASSED: API key configured correctly\n")

---
## Test 4: API Connectivity

Tests actual connection to Anthropic API and validates your API key works.

**This test makes a real API call** (costs ~$0.001)

In [ ]:
from google.colab import userdata
import anthropic

print("Testing API connectivity...\n")

try:
    # Get API key
    api_key = userdata.get('ANTHROPIC_API_KEY')
    
    # Create client
    client = anthropic.Anthropic(api_key=api_key)
    print("✓ API client initialized")
    
    # Test API call
    message = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=50,
        messages=[
            {
                "role": "user",
                "content": "Respond with exactly: 'API connection successful'"
            }
        ]
    )
    
    response_text = message.content[0].text
    print(f"✓ API call successful")
    print(f"✓ Claude response: {response_text}")
    
    # Verify usage tracking
    input_tokens = message.usage.input_tokens
    output_tokens = message.usage.output_tokens
    print(f"✓ Token usage tracked: {input_tokens} input, {output_tokens} output")
    
except anthropic.AuthenticationError:
    print("✗ Authentication failed")
    print("\nPossible causes:")
    print("1. Invalid API key")
    print("2. API key expired or revoked")
    print("3. Billing issue (payment method declined)")
    print("\nActions:")
    print("1. Verify key at: https://console.anthropic.com/settings/keys")
    print("2. Check billing: https://console.anthropic.com/settings/billing")
    print("3. Regenerate key if needed")
    sys.exit(1)

except anthropic.RateLimitError:
    print("✗ Rate limit exceeded")
    print("\nThis is unusual for a test call. Possible causes:")
    print("1. Too many recent API calls")
    print("2. Free tier limits exhausted")
    print("\nWait 60 seconds and re-run this cell")
    sys.exit(1)

except anthropic.APIError as e:
    print(f"✗ API error: {e}")
    print("\nCheck Anthropic status: https://status.anthropic.com")
    print("If service is operational, contact instructor")
    sys.exit(1)

except Exception as e:
    print(f"✗ Unexpected error: {e}")
    print("\nContact instructor: javier@jmarin.info")
    sys.exit(1)

print("\n✓ Test 4 PASSED: API connectivity verified\n")

---
## Test 5: LangChain Agent Execution

Tests a complete LangChain workflow with Claude.

This simulates the type of agent interactions you'll build in the course.

In [ ]:
from google.colab import userdata
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage, SystemMessage

print("Testing LangChain agent execution...\n")

try:
    # Get API key
    api_key = userdata.get('ANTHROPIC_API_KEY')
    
    # Initialize LangChain ChatAnthropic
    llm = ChatAnthropic(
        model="claude-sonnet-4-20250514",
        anthropic_api_key=api_key,
        max_tokens=100,
        temperature=0
    )
    print("✓ LangChain model initialized")
    
    # Create messages
    messages = [
        SystemMessage(content="You are a helpful assistant. Be concise."),
        HumanMessage(content="What is 7 * 8? Reply with just the number.")
    ]
    
    # Invoke model
    response = llm.invoke(messages)
    print(f"✓ Agent invoked successfully")
    print(f"✓ Agent response: {response.content}")
    
    # Verify response format
    if hasattr(response, 'content'):
        print("✓ Response format correct")
    else:
        print("✗ Unexpected response format")
        print(f"  Got: {type(response)}")
        sys.exit(1)
    
    # Verify response contains expected answer
    if "56" in response.content:
        print("✓ Agent computation correct (7 * 8 = 56)")
    else:
        print(f"⚠ Unexpected answer: {response.content}")
        print("  This may indicate model behavior issues, but connection is working")
    
except Exception as e:
    print(f"✗ LangChain execution failed: {e}")
    print("\nThis should not happen if previous tests passed.")
    print("Contact instructor: javier@jmarin.info")
    sys.exit(1)

print("\n✓ Test 5 PASSED: LangChain agent working correctly\n")

---
## Test 6: Callback System (Observability Preview)

Tests LangChain's callback system, which we'll use extensively for monitoring agents.

In [ ]:
from google.colab import userdata
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage
from langchain.callbacks import StdOutCallbackHandler

print("Testing callback system...\n")

try:
    # Get API key
    api_key = userdata.get('ANTHROPIC_API_KEY')
    
    # Initialize model with callback
    llm = ChatAnthropic(
        model="claude-sonnet-4-20250514",
        anthropic_api_key=api_key,
        max_tokens=50,
        callbacks=[StdOutCallbackHandler()]
    )
    print("✓ Callback handler attached")
    
    # Make call with callback
    print("\n--- Callback Output (this is what observability looks like) ---")
    response = llm.invoke([HumanMessage(content="Say 'callbacks working'")])
    print("--- End Callback Output ---\n")
    
    print("✓ Callback system functional")
    print("✓ You'll use this for monitoring in Chapter 2")
    
except Exception as e:
    print(f"✗ Callback test failed: {e}")
    print("\nThis is not critical but indicates potential issues with observability features.")
    print("Mention this to instructor at start of course.")

print("\n✓ Test 6 PASSED: Callback system working\n")

---
## Verification Summary

If you see this cell, all tests passed! Run the cell below for final confirmation.

In [ ]:
from datetime import datetime

print("="*60)
print("SETUP VERIFICATION COMPLETE")
print("="*60)

print("\nAll tests passed successfully!\n")

print("✓ Test 1: Google Colab environment")
print("✓ Test 2: Package installation")
print("✓ Test 3: API key configuration")
print("✓ Test 4: API connectivity")
print("✓ Test 5: LangChain agent execution")
print("✓ Test 6: Callback system")

print("\n" + "="*60)
print("YOU ARE READY FOR DAY 1")
print("="*60)

print(f"\nVerification completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\nNext steps:")
print("1. Review the course calendar: https://github.com/Javihaus/agents_observability_bootcamp/blob/main/CALENDAR.md")
print("2. Read Chapter 1 content before Day 1")
print("3. Join the Google Meet session on time")

print("\nQuestions? Email: javier@jmarin.info")
print("\n" + "="*60)

---
## Troubleshooting Guide

### If Any Test Failed

**Test 1 (Environment)**
- Ensure you opened this notebook in Google Colab
- URL should start with `colab.research.google.com`
- Try: Runtime → Restart runtime

**Test 2 (Packages)**
- Check internet connectivity
- Try: Runtime → Restart runtime, then re-run installation cell
- If persistent, contact instructor

**Test 3 (API Key)**
- Follow setup guide: [setup/01_claude_api_setup.md](../setup/01_claude_api_setup.md)
- Verify key name is exactly `ANTHROPIC_API_KEY` (case-sensitive)
- Check "Notebook access" toggle is ON
- Try: Runtime → Restart runtime after saving secret

**Test 4 (Connectivity)**
- Check Anthropic status: https://status.anthropic.com
- Verify billing: https://console.anthropic.com/settings/billing
- Regenerate API key if authentication fails
- Ensure payment method is active

**Test 5 (LangChain)**
- If this fails but Test 4 passed, contact instructor
- This indicates LangChain integration issue

**Test 6 (Callbacks)**
- Not critical for Day 1
- Mention to instructor if it fails

---

### Still Having Issues?

**Before contacting instructor**, gather this information:
1. Which test failed?
2. Complete error message (copy/paste)
3. Screenshot of the error
4. Have you completed [PREREQUISITES.md](../PREREQUISITES.md)?

**Contact**: javier@jmarin.info

**Subject line**: "Setup Verification Failed - Test X"

---

### API Cost Tracking

This verification notebook costs approximately:
- Test 4: ~$0.001
- Test 5: ~$0.001
- Test 6: ~$0.001
- **Total**: ~$0.003 (less than one cent)

Track your usage: https://console.anthropic.com/settings/usage